In [4]:
# -*- coding: utf-8 -*-
import copy
import itertools

def preprocessing(filename):
    with open(filename + '.txt') as file:

        #Preprocessing input attributes by adding Attribute number
        genes = []
        data = {}
        mas_dic = {}
        items_list = set()
        for line in file.readlines():
            line = line.strip()
            entry = line.split("\t")

            i=0
            t = []
            for ex in entry:

                if i+1<len(entry):
                    ex = "G" + str(i+1) + "_" + ex
        #Creating a Unique List of all the Candidate items present in the data
                items_list.add(ex)
                t.append(ex)
                i=i+1
            genes.append(t)
            
    return genes,items_list,mas_dic
        

    
#Support Calculation for all the Candidate itemsets present

def support_calculation(genes,candidates_list,mas_dic,total_entries):
    
    for entry in genes:
        for cand in candidates_list:
            c_list = cand.split(",")
            if(set(c_list).issubset(set(entry))):
                mas_dic[cand] += 1
    
    




#Generation of all possible combinations of  frequent itemsets of all lengths for the given support threshold

def frequent_itemset(support_threshold,genes,items_list):
    number_of_entries = len(genes)
    support_threshold = support_threshold/100
    candidates_list = list(items_list)
    ans=[]
    freq_itemset = {}
    datas = {}
    temp_list = candidates_list
    total_count = 0
    for itemset_length in range(1,len(items_list)+1):

        if itemset_length == 1:
            for item in candidates_list:
                mas_dic[item] = 0
        else:

            for j in range(0,len(candidates_list)):
                item1 = candidates_list[j].split(",")
                for k in range(j+1,len(candidates_list)):
                    item2 = candidates_list[k].split(",")
                    combined_list = item1 + item2
                    final = list(set(combined_list))
                    item_len = len(final)

                    if(item_len == itemset_length):
                        word = ""
                        word = ",".join(sorted(final))
                        ans.append(word)
                        mas_dic[word] = 0
                        temp_list.append(word)

        temp_list = list(set(temp_list))
        candidates_list = temp_list
        temp_list = []
        support_calculation(genes,candidates_list,mas_dic,number_of_entries)

        
        final_list = {}

        for item in candidates_list:
            if mas_dic[item]/number_of_entries >= support_threshold and len(item.split(",")) == itemset_length:
                final_list[item]= mas_dic[item]
        datas[itemset_length]=final_list        
        if len(final_list) == 0:
            break

        freq_itemset[itemset_length] = list(final_list.keys())
        candidates_list = list(final_list.keys())  
        total_count += len(candidates_list)
        print("\n" + "Number of frequent itemsets of length-" + str(itemset_length) + " : " +  str(len(candidates_list)))
        
    data = datas
    print("\n" + "Total Number of frequent itemsets : " + str(total_count))
    return datas
    


In [5]:
def support(body_list,genes):
    count = 0
    
    for entry in genes:
        if(set(body_list).issubset(entry)):
            count += 1
    
    return count




#Generating all the rules satisfying the given confidence threshold

def generating_rules(confidence_threshold,data,genes,mas_dic):
    rules = set()
    for itemset_length in data.keys():
        if itemset_length > 1:
            for itemset in data[itemset_length]:
                head_count = mas_dic[itemset]
                items = itemset.split(",")
                if head_count != 0:
                    for body_length in range(len(items)-1,0,-1):
                        for body in list(itertools.combinations(items,body_length)):
                            body_list = list(body)
                            head = list(set(body_list)^set(items))
                            rule = ",".join(head) + "->" + ",".join(body_list)
                            if rule not in rules:
                                body_count = support(body_list,genes)
                                confidence = (head_count/body_count) * 100
                                if(confidence >= confidence_threshold):
                                    rules.add(rule)
    return rules

In [2]:
#Template 1 satisfying rules Filtering Process for a Given Support And Confidence Threshold


def template1_input(rules):
    inputs = input("Please enter the inputs in the following format:  RULE|HEAD|BODY;ANY|NONE|1;item1,item2.." + "\n" + "An Example Input is : " +"\n"+ "RULE;NONE;G59_Up,G10_Down,G98_Up ").split(";")
    answer = template1(rules,inputs)
    return answer


def template1(rules,inputs):
    query = inputs[0].upper()
    condition = inputs[1].upper()
    items_list = inputs[2].split(",")
    answer_rules = set()
    
    for rule in rules:
        rule_list = rule.split("->")
        rule_head = set(rule_list[1].split(","))
        rule_body = set(rule_list[0].split(","))
        items_list_set = set(items_list)
        if condition == 'ANY':
            if query == 'RULE' and len((rule_head | rule_body) & items_list_set) > 0:
                answer_rules.add(rule)
            elif query == 'HEAD' and len(rule_head & items_list_set) > 0:
                answer_rules.add(rule)
            elif query == 'BODY' and len(rule_body & items_list_set) > 0:
                answer_rules.add(rule)
                
        elif condition == 'NONE':
            if query == 'RULE' and len((rule_head | rule_body) & items_list_set) == 0:
                answer_rules.add(rule)
            elif query == 'HEAD' and len(rule_head & items_list_set) == 0:
                answer_rules.add(rule)
            elif query == 'BODY' and len(rule_body & items_list_set) == 0:
                answer_rules.add(rule)
                
        elif condition.isdigit():
            lengths = int(condition)
            if query == 'RULE' and len((rule_head | rule_body) & items_list_set) == lengths:
                answer_rules.add(rule)
            elif query == 'HEAD' and len(rule_head & items_list_set) == lengths:
                answer_rules.add(rule)
            elif query == 'BODY' and len(rule_body & items_list_set) == lengths:
                answer_rules.add(rule)
    
    return list(answer_rules)

In [6]:
#Template 2 satisfying rules Filtering Process for a Given Support And Confidence Threshold

def template2_input(rules):
    inputs = input("Please enter the inputs in the following format:  RULE|HEAD|BODY;length" + "\n" + "An Example Input is : " "\n"+ "RULE;2").split(";")
    answer = template2(rules,inputs)
    return answer
    
    
    
    
def template2(rules,inputs):
    query = inputs[0].upper()
    length = int(inputs[1])
    answer_rules = set()
    for rule in rules:
        rule_list = rule.split("->")
        rule_head = set(rule_list[1].split(","))
        rule_body = set(rule_list[0].split(","))
        if query == "RULE" and len(rule_head | rule_body) >= length:
             answer_rules.add(rule)
        elif query == "HEAD" and len(rule_head) >= length:
            answer_rules.add(rule)
        elif query == "BODY" and len(rule_body) >= length:
            answer_rules.add(rule)
    
    return list(answer_rules)

In [7]:
#Template 3 satisfying rules Filtering Process for a Given Support And Confidence Threshold

def template3(rules):
    inputs = input("Please enter the inputs in the following format: template_choice_1,Operator,template_choice_2|template_choice_1_input|template_choice_2_input" + "\n" + "An Example Input is : " + "\n" + "1and2|RULE;NONE;G59_Up,G10_Down|HEAD;2 ").split("|")
    template = inputs[0]
    operator = ""
    if len(template) == 4:
        templates = template.lower().split("or")
        operator = "OR"
    elif len(template) == 5:
        templates = template.lower().split("and")
        operator = "AND"
    input_1 = inputs[1].split(";")
    input_2 = inputs[2].split(";")
    answer_1 = set()
    answer_2 = set()
    if templates[0] == '1':
        answer_1 = template1(rules,input_1)
    elif templates[0] == '2':
        answer_1 = template2(rules,input_1)
    
    if templates[1] == '1':
        answer_2 = template1(rules,input_2)
    elif templates[1] == '2':
        answer_2 = template2(rules,input_2)
        
    if operator.upper() == 'OR':
        final_answer = (set(answer_1) | set(answer_2))
    
    elif operator.upper() == 'AND':
        final_answer = (set(answer_1) & set(answer_2))
    
    return final_answer

In [ ]:

#Entering the file name 
filename = input("Enter the file name")
#Preprocessing the file data
genes,items_list,mas_dic = preprocessing(filename)
#Entering the Support threshold
a = input("Enter the Support Percentage Threshold : ")
#Generating all Frequent Itemsets
data = frequent_itemset(int(a),genes,items_list)
#Entering the Confidence Threshold
b = input("\n" + "Enter the Confidence Percentage Threshold : ")
#Generating all the rules satisfying the confidence threshold
rules = generating_rules(int(b),data,genes,mas_dic)
print("\n" + "The Total Number of Rules Generated are : " + str(len(rules)))
print("\n" + "The rules are :")
print(rules)
while(True):
    a = input("\n" + "Please enter the template choice to continue or EXIT to break")
    if a.upper() == "EXIT":
        break
    elif a == "1":
        ans = template1_input(rules)
        print("\n" + "The Total Number of Rules : " + str(len(ans)))
        if len(ans) == 0:
            print("There are No Rules")
        else:
            print("\n" + "The Rules are : " + "\n")
            print(ans)
    elif a == "2":
        ans = template2_input(rules)
        print("\n" + "The Total Number of Rules are : " + str(len(ans)) + "\n")
        if len(ans) == 0:
            print("There are No Rules")
        else:
            print("The rules are :" + "\n")
            print(ans)
    elif a == "3":
        ans = template3(rules)
        print("\n" + "The Number of Rules : " + str(len(ans)) + "\n")
        if len(ans) == 0:
            print("There are No Rules")
        else:
            print("The rules are : " + "\n")
            print(ans)
    

Enter the file nameassociation-rule-test-data
Enter the Support Percentage Threshold : 50

Number of frequent itemsets of length-1 : 109

Number of frequent itemsets of length-2 : 63

Number of frequent itemsets of length-3 : 2

Total Number of frequent itemsets : 174

Enter the Confidence Percentage Threshold : 70

The Total Number of Rules Generated are : 117

The rules are :
{'G96_Down->G59_Up,G72_Up', 'G88_Down->G87_Up', 'G10_Down->G94_Up', 'G88_Down->G54_Up', 'G88_Down->G10_Down', 'G38_Down->G41_Down', 'G38_Down->G88_Down', 'G52_Down->G38_Down', 'G82_Down->G72_Up', 'G59_Up->G87_Up', 'G38_Down->G65_Down', 'G59_Up->G88_Down', 'G88_Down->G8_Up', 'G88_Down->G28_Down', 'G59_Up->G72_Up,G82_Down', 'G38_Down->G67_Up', 'G38_Down->G32_Down', 'G28_Down->G47_Up', 'G67_Up->G1_Up', 'G82_Down->G13_Down', 'G54_Up->G24_Down', 'G28_Down->G87_Up', 'G1_Up->G54_Up', 'G72_Up->G97_Down', 'G32_Down->G38_Down', 'G10_Down->G70_Down', 'G28_Down->G32_Down', 'G70_Down->G10_Down', 'G10_Down->G28_Down', 'G97_Do


Please enter the template choice to continue or EXIT to breakHEAD;1;G59_Up,G10_Down

Please enter the template choice to continue or EXIT to break1
Please enter the inputs in the following format:  RULE|HEAD|BODY;ANY|NONE|1;item1,item2..
An Example Input is : 
RULE;NONE;G59_Up,G10_Down,G98_Up HEAD;1;G59_Up,G10_Down

The Total Number of Rules : 17

The Rules are : 

['G1_Up->G10_Down', 'G72_Up->G59_Up', 'G96_Down->G59_Up,G72_Up', 'G82_Down->G59_Up,G72_Up', 'G70_Down->G10_Down', 'G96_Down->G59_Up', 'G72_Up->G59_Up,G96_Down', 'G47_Up->G10_Down', 'G59_Up->G10_Down', 'G38_Down->G10_Down', 'G13_Down->G59_Up', 'G88_Down->G10_Down', 'G94_Up->G10_Down', 'G88_Down->G59_Up', 'G72_Up->G59_Up,G82_Down', 'G82_Down->G59_Up', 'G28_Down->G10_Down']

Please enter the template choice to continue or EXIT to breakBODY;ANY;G59_Up

Please enter the template choice to continue or EXIT to break1
Please enter the inputs in the following format:  RULE|HEAD|BODY;ANY|NONE|1;item1,item2..
An Example Input is : 
RU


Please enter the template choice to continue or EXIT to break3
Please enter the inputs in the following format: template_choice_1,Operator,template_choice_2|template_choice_1_input|template_choice_2_input
An Example Input is : 
1and2|RULE;NONE;G59_Up,G10_Down|HEAD;2 1or2|HEAD;ANY;G10_Down|BODY;1;G59_Up

The Number of Rules : 117

The rules are : 

{'G96_Down->G59_Up,G72_Up', 'G88_Down->G87_Up', 'G13_Down->G72_Up', 'G10_Down->G47_Up', 'G10_Down->G94_Up', 'G88_Down->G54_Up', 'G47_Up->G10_Down', 'G96_Down->G59_Up', 'G10_Down->G38_Down', 'G72_Up->G32_Down', 'G88_Down->G10_Down', 'G82_Down->G97_Down', 'G38_Down->G41_Down', 'G38_Down->G88_Down', 'G52_Down->G38_Down', 'G82_Down->G72_Up', 'G32_Down->G6_Up', 'G59_Up->G87_Up', 'G38_Down->G65_Down', 'G38_Down->G52_Down', 'G59_Up->G88_Down', 'G88_Down->G8_Up', 'G13_Down->G59_Up', 'G59_Up->G38_Down', 'G59_Up->G96_Down', 'G6_Up->G28_Down', 'G88_Down->G28_Down', 'G59_Up->G72_Up,G82_Down', 'G70_Down->G38_Down', 'G41_Down->G28_Down', 'G82_Down->G59_Up